# Qhue experiments

Experiments with the [Qhue](https://github.com/quentinsf/qhue) python module.

If you haven't already, then `pip install qhue` before starting.  

Some of these examples may assume you have a recent bridge with recent software.

*If you're viewing this with my sample output, I've truncated some of it for readability. I have a lot of lights!*

## Basics 

In [15]:
# Put in the IP address of your Hue bridge here
BRIDGE_IP='192.168.1.185'
username='BykNjzzgLxSgbHS5AfCMkSiLFrAVG4mHxxm1yFTp'

from qhue import Bridge, QhueException, create_new_username
import time
from random import randint

In [3]:
bridge = Bridge(BRIDGE_IP, username)
lights = bridge.lights()
for num, info in lights.items():
    print("{:16} {}".format(info['name'], num))

Table            1
Bar              2
Hue color lamp 3 3
Hue color lamp 4 4


Let's get the numbers and names of the lights:

In [5]:
bridge.lights[1].state(on=True)

[{'success': {'/lights/1/state/on': True}}]

In [37]:
seq = [
    { 'light': 1, 'state': True, 'hue': randint(200, 65535), 'br': randint(0, 255) },
    { 'light': 2, 'state': True, 'hue': randint(4381, 65535), 'br': randint(0, 255) },
    { 'light': 1, 'state': True, 'hue': randint(9192, 65535), 'br': randint(0, 255) },
    { 'light': 4, 'state': True, 'hue': randint(7141, 65535), 'br': randint(0, 255) },
    { 'light': 2, 'state': True, 'hue': randint(0, 65535), 'br': randint(0, 255) },
    { 'light': 4, 'state': True, 'hue': randint(4000, 65535), 'br': randint(0, 255) },
]


In [39]:
for s in seq:
    bridge.lights[s['light']].state(on=s['state'], hue=s['hue'], bri=255)
    time.sleep(1)

In [41]:
while True:
    bridge.lights[randint(1,4)].state(on=True, hue=randint(200, 65535), bri=255)
    time.sleep(1)

KeyboardInterrupt: 

Let's try interactively changing a light.  You could make this a lot more sophisticated:

In [16]:
from ipywidgets import interact, interactive, fixed
import ipywidgets as widgets

def setlight(lightid='14', on=True, ct=128, bri=128):
    bridge.lights[lightid].state(on=on)
    if on:
        bridge.lights[lightid].state(bri=bri, ct=ct)

light_list = interact(setlight,
                      lightid = widgets.Dropdown(
                            options={ lights[i]['name']:i for i in lights },
                            value='14',
                            description='Light:',
                        ),
                      on = widgets.Checkbox(value=True, description='On/off'),
                      bri = widgets.IntSlider(min=0,max=255,value=128, description='Bright:'),
                      ct = widgets.IntSlider(min=0,max=255,value=128, description='Colour:'))

TraitError: Invalid selection: value not found

The [YAML format](https://en.wikipedia.org/wiki/YAML) is a nice way to view the sometimes large amount of structured information which comes back from the bridge. 

If you haven't got the Python yaml module, `pip install PyYAML`.

In [26]:
import yaml
print("{} lights:\n".format(len(lights)))
print(yaml.safe_dump(lights, indent=4))

4 lights:

'1':
    capabilities:
        certified: true
        control:
            colorgamut:
            - [0.6915, 0.3083]
            - [0.17, 0.7]
            - [0.1532, 0.0475]
            colorgamuttype: C
            ct: {max: 500, min: 153}
            maxlumen: 800
            mindimlevel: 1000
        streaming: {proxy: true, renderer: true}
    config: {archetype: sultanbulb, direction: omnidirectional, function: mixed}
    manufacturername: Philips
    modelid: LCT014
    name: Table
    productid: Philips-LCT014-1-A19ECLv4
    productname: Hue color lamp
    state:
        alert: none
        bri: 198
        colormode: hs
        ct: 347
        effect: none
        hue: 57161
        mode: homeautomation
        'on': true
        reachable: true
        sat: 254
        xy: [0.4469, 0.1898]
    swconfigid: 3027B5DE
    swupdate: {lastinstall: null, state: noupdates}
    swversion: 1.29.0_r21169
    type: Extended color light
    uniqueid: 00:17:88:01:02:8e:a9:d2-0b

In [13]:
print(yaml.safe_dump(bridge.lights['3'](), indent=4))

capabilities:
    certified: true
    control:
        colorgamut:
        - [0.6915, 0.3083]
        - [0.17, 0.7]
        - [0.1532, 0.0475]
        colorgamuttype: C
        ct: {max: 500, min: 153}
        maxlumen: 800
        mindimlevel: 1000
    streaming: {proxy: true, renderer: true}
config: {archetype: sultanbulb, direction: omnidirectional, function: mixed}
manufacturername: Philips
modelid: LCT014
name: Hue color lamp 3
productid: Philips-LCT014-1-A19ECLv4
productname: Hue color lamp
state:
    alert: none
    bri: 210
    colormode: hs
    ct: 500
    effect: none
    hue: 10012
    mode: homeautomation
    'on': true
    reachable: false
    sat: 254
    xy: [0.3803, 0.2402]
swconfigid: 3027B5DE
swupdate: {lastinstall: null, state: noupdates}
swversion: 1.29.0_r21169
type: Extended color light
uniqueid: 00:17:88:01:02:dc:aa:6f-0b



## Scenes 

Let's look at the scenes defined in the bridge, and their IDs.  Some of these may be created manually, and others by the Hue app or other software.

Version 1-type scenes just refer to the lights - each light is told: "Set the value you have stored for this scene".

Version 2 scenes have more details stored in the hub, which is generally more useful.

In [14]:
scenes = bridge.scenes()
print("{} scenes:\n".format(len(scenes)))
print(yaml.safe_dump(scenes, indent=4))

15 scenes:

-rhq90wNuLwkZd1:
    appdata: {data: v46L0_r01_d01, version: 1}
    lastupdated: '2018-08-17T02:05:01'
    lights: ['1', '2', '3', '4']
    locked: false
    name: Relax
    owner: 8Oe9SokBWRkm8MzD1Qozv9rN90LB6Q2e-b0wW62e
    picture: ''
    recycle: false
    version: 2
8GQMEfegZ55UCvU:
    appdata: {data: fF3By_r01_d17, version: 1}
    lastupdated: '2018-08-17T02:05:01'
    lights: ['1', '2', '3', '4']
    locked: false
    name: Arctic aurora
    owner: 8Oe9SokBWRkm8MzD1Qozv9rN90LB6Q2e-b0wW62e
    picture: ''
    recycle: false
    version: 2
A8gNrfpotn991uv:
    appdata: {data: 4GDwe_r01_d16, version: 1}
    lastupdated: '2018-08-17T02:05:01'
    lights: ['1', '2', '3', '4']
    locked: false
    name: Tropical twilight
    owner: 8Oe9SokBWRkm8MzD1Qozv9rN90LB6Q2e-b0wW62e
    picture: ''
    recycle: false
    version: 2
Dr8VReLcesoErd3:
    appdata: {data: NLpwm_r01, version: 1}
    lastupdated: '2018-08-17T03:11:42'
    lights: ['1', '2', '3', '4']
    locked: false
  

Details of a particular scene from the list:

In [27]:
print(yaml.safe_dump(bridge.scenes['8GQMEfegZ55UCvU']()))

appdata: {data: fF3By_r01_d17, version: 1}
lastupdated: '2018-08-17T02:05:01'
lights: ['1', '2', '3', '4']
lightstates:
  '1':
    bri: 137
    'on': true
    xy: [0.139, 0.081]
  '2':
    bri: 137
    'on': true
    xy: [0.1715, 0.3528]
  '3':
    bri: 137
    'on': true
    xy: [0.1712, 0.2155]
  '4':
    bri: 137
    'on': true
    xy: [0.139, 0.081]
locked: false
name: Arctic aurora
owner: 8Oe9SokBWRkm8MzD1Qozv9rN90LB6Q2e-b0wW62e
picture: ''
recycle: false
version: 2



Let's list scenes with IDs, last updated time, and the lights affected:

In [28]:
for sid, info in scenes.items():
    print("\n{:16} {:20} {}".format( sid, info['name'], info['lastupdated']))
    for li in info['lights']:
        print("{:40}- {}".format('', lights[li]['name']))


jgxkUGd0oUXENhV  Savanna sunset       2018-08-17T02:05:00
                                        - Table
                                        - Bar
                                        - Hue color lamp 3
                                        - Hue color lamp 4

A8gNrfpotn991uv  Tropical twilight    2018-08-17T02:05:01
                                        - Table
                                        - Bar
                                        - Hue color lamp 3
                                        - Hue color lamp 4

8GQMEfegZ55UCvU  Arctic aurora        2018-08-17T02:05:01
                                        - Table
                                        - Bar
                                        - Hue color lamp 3
                                        - Hue color lamp 4

NReux9VZOyzFaAu  Spring blossom       2018-08-17T02:05:01
                                        - Table
                                        - Bar
                                  

Tidying things up; let's delete a scene:

In [62]:
# Uncomment and edit this if you actually want to run it!
# print(bridge.scenes['cd06c70f7-on-0'](http_method='delete'))

Show the details of the scenes that affect a particular light:

In [63]:
lightname = 'Sitting room 1'
# How's this for a nice use of python iterators?
light_id = next(i for i,info in lights.items() if info['name'] == lightname)
print("Light {} - {}".format(light_id, lightname))
for line in ["{} : {:20} {}".format(sid, info['name'], info['lastupdated']) for sid, info in scenes.items() if light_id in info['lights']]:
    print(line)

Light 21 - Sitting room 1
SePln7Lt9-H7Hm7 : Bright               2017-02-01T09:52:06
VaknVPkUZnSrdiB : Bright               2017-02-01T07:54:30
3owQUn01W7nVsxR : Evening              2017-02-01T07:54:29
GYOWpf6lHjaVc3T : Off                  2016-09-13T21:11:19
wG25IXpcHHTim4g : Off                  2017-02-01T07:54:30
SUThT3XiV7sSzml : Warm                 2017-02-01T07:54:30
KZNM2DZmdcydRIc : All warm             2016-08-06T23:50:55
IB57U3scrj4cQWk : Read                 2017-02-01T07:54:29
wVXtOrFmdnySqUz : Dimmed               2017-02-01T07:54:30
YDfVlYFWoaL6yv5 : Nightlight           2017-02-01T07:54:29


## Groups and rooms

Let's look at groups:

In [64]:
print(yaml.safe_dump(bridge.groups(), indent=4))

'1':
    action:
        alert: none
        bri: 240
        colormode: xy
        ct: 343
        effect: none
        hue: 15360
        'on': false
        sat: 119
        xy: [0.4436, 0.4062]
    lights: ['3', '6']
    name: Kitchen
    recycle: false
    state: {all_on: false, any_on: false}
    type: LightGroup
'2':
    action:
        alert: none
        bri: 126
        colormode: xy
        ct: 267
        effect: none
        hue: 16528
        'on': false
        sat: 29
        xy: [0.3944, 0.385]
    lights: ['7', '8']
    name: Hall
    recycle: false
    state: {all_on: false, any_on: false}
    type: LightGroup
'3':
    action:
        alert: none
        bri: 240
        colormode: xy
        ct: 343
        effect: none
        hue: 15360
        'on': false
        sat: 119
        xy: [0.4436, 0.4062]
    lights: ['12', '11', '6', '3']
    name: Dimmer 11
    recycle: false
    state: {all_on: false, any_on: false}
    type: LightGroup
...



The current Hue software creates 'rooms', which are groups with a type value set to Room:

In [65]:
groups = bridge.groups()
rooms = [(gid, info['name']) for gid, info in groups.items() if info.get('type') == 'Room' ]
for room_id, info in rooms:
    print("{:3} : {}".format(room_id, info))

4   : Kitchen
5   : Garden
6   : Sitting room
7   : Hall
8   : Landing
9   : Bedroom


## Sensors

Sensors are mostly switches, but a few other things come under the same category in the bridge.  There's a 'daylight' sensor, implemented in software, for example, and various bits of state can also be stored here so they can be used in rule conditions later.

In [66]:
sensors = bridge.sensors()
summary = [(info['name'], i, info['type']) for i,info in sensors.items()]
# Sort by name
# Python 2: summary.sort(lambda a,b: cmp(a[0], b[0]))
# Python 3:
summary.sort(key = lambda a: a[0])
for n,i,t in summary:
    print("{:30} {:>3} {}".format(n,i,t))
    #print(bridge.sensors[i]())
    

Bedroom tap                      7 ZGPSwitch
Daylight                         1 Daylight
Dimmer Switch 11 SceneCycle     14 CLIPGenericStatus
Dimmer Switch 12 SceneCycle     13 CLIPGenericStatus
Dining Room                      9 ZGPSwitch
Hall                             8 ZGPSwitch
Hall dimmer                     12 ZLLSwitch
Hall sensor                     24 ZLLPresence
Hue ambient light sensor 1      17 ZLLLightLevel
Hue ambient light sensor 2      21 ZLLLightLevel
Hue ambient light sensor 3      25 ZLLLightLevel
Hue ambient light sensor 4      29 ZLLLightLevel
Hue temperature sensor 1        15 ZLLTemperature
Hue temperature sensor 2        19 ZLLTemperature
Hue temperature sensor 3        23 ZLLTemperature
Hue temperature sensor 4        27 ZLLTemperature
Kitchen dimmer                  11 ZLLSwitch
Kitchen tap                      2 ZGPSwitch
Landing sensor                  16 ZLLPresence
Landing tap                      3 ZGPSwitch
Laundry sensor                  28 ZLLPresenc

Here's a more complete list:

In [67]:
print(yaml.safe_dump(bridge.sensors(), indent=4))

'1':
    config: {configured: false, 'on': true, sunriseoffset: 30, sunsetoffset: -30}
    manufacturername: Philips
    modelid: PHDL00
    name: Daylight
    state: {daylight: null, lastupdated: none}
    swversion: '1.0'
    type: Daylight
'10':
    config: {'on': true}
    manufacturername: Philips
    modelid: ZGPSWITCH
    name: Sitting room
    state: {buttonevent: 34, lastupdated: '2017-07-04T22:03:46'}
    swupdate: {lastinstall: null, state: notupdatable}
    type: ZGPSwitch
    uniqueid: 00:00:00:00:00:41:1f:34-f2
'11':
    config:
        battery: 84
        'on': true
        pending: []
        reachable: true
    manufacturername: Philips
    modelid: RWL021
    name: Kitchen dimmer
    state: {buttonevent: 4002, lastupdated: '2017-07-04T09:42:08'}
    swupdate: {lastinstall: null, state: noupdates}
    swversion: 5.45.1.17846
    type: ZLLSwitch
    uniqueid: 00:17:88:01:10:33:28:66-02-fc00
...



## Rules

Rules map sensor events etc. to actions.


In [68]:
rules = bridge.rules()
print(yaml.safe_dump(rules, indent=4))


'1':
    actions:
    -   address: /groups/4/action
        body: {scene: HfANai28yTRy07O}
        method: PUT
    conditions:
    - {address: /sensors/2/state/lastupdated, operator: dx}
    - {address: /sensors/2/state/buttonevent, operator: eq, value: '18'}
    created: '2016-09-23T09:10:49'
    lasttriggered: '2017-07-04T20:50:01'
    name: Tap 2.4
    owner: IneFZ4CIEdSQQN4oCGExhsi0cWquxMrZY6tEElKM
    recycle: false
    status: enabled
    timestriggered: 3
'10':
    actions:
    -   address: /groups/8/action
        body: {scene: zvuMOXo8vmShFZK}
        method: PUT
    conditions:
    - {address: /sensors/4/state/lastupdated, operator: dx}
    - {address: /sensors/4/state/buttonevent, operator: eq, value: '18'}
    created: '2016-07-23T11:48:50'
    lasttriggered: none
    name: Tap 4.4
    owner: IneFZ4CIEdSQQN4oCGExhsi0cWquxMrZY6tEElKM
    recycle: false
    status: enabled
    timestriggered: 0
...



Show the rules triggered by the Sitting Room switch.

For Tap switches, buttons 1,2,3,4 are represented by the values 34,16,17,18 respectively.

In [69]:
switch = '10'  # sitting room
print("Switch {} -- {}\n".format(switch, sensors[switch]['name']))

# State changes on the switch will look like this:
state_string = "/sensors/{}/state/".format(switch)

# Look through the rules for once which contain this 
# string in their conditions:
for rid, info in rules.items():
    this_switch = False
    matching_conditions = [c for c in info['conditions'] if state_string in c['address']]
    if len(matching_conditions) > 0:
        print("{:3} {:20}".format(rid, info['name']))
        for c in info['conditions']:
            print("   ? condition {}".format(c))
        for a in info['actions']:

            # If the action involves applying a scene, get its name
            scene_name = ""
            if 'scene' in a['body']:
                scene_name = scenes[a['body']['scene']]['name']
            
            print("   - action address {} body {!s:29s} {} ".format( a['address'], a['body'], scene_name))
            

Switch 10 -- Sitting room

29  Tap 10.3            
   ? condition {'address': '/sensors/10/state/lastupdated', 'operator': 'dx'}
   ? condition {'address': '/sensors/10/state/buttonevent', 'operator': 'eq', 'value': '17'}
   - action address /groups/6/action body {'scene': 'SUThT3XiV7sSzml'}  Warm 
30  Tap 10.2            
   ? condition {'address': '/sensors/10/state/lastupdated', 'operator': 'dx'}
   ? condition {'address': '/sensors/10/state/buttonevent', 'operator': 'eq', 'value': '16'}
   - action address /groups/6/action body {'scene': 'SePln7Lt9-H7Hm7'}  Bright 
31  Tap 10.4            
   ? condition {'address': '/sensors/10/state/lastupdated', 'operator': 'dx'}
   ? condition {'address': '/sensors/10/state/buttonevent', 'operator': 'eq', 'value': '18'}
   - action address /groups/6/action body {'scene': '3owQUn01W7nVsxR'}  Evening 
32  2:huelabs/tap-toggle
   ? condition {'address': '/sensors/10/state/buttonevent', 'operator': 'eq', 'value': '34'}
   ? condition {'address': '

Let's see what is actually done by one of these scenes:

In [70]:
scene='3owQUn01W7nVsxR' # 'Evening' scene button 10.4

s = bridge.scenes[scene]()
print(yaml.safe_dump(s, indent=4))

appdata: {data: mdVDQ_r06_d99, version: 1}
lastupdated: '2017-02-01T07:54:29'
lights: ['14', '21', '32', '33']
lightstates:
    '14':
        bri: 189
        'on': true
        xy: [0.5102, 0.3642]
    '21': {bri: 189, 'on': true}
    '32': {bri: 189, 'on': true}
    '33': {bri: 189, 'on': true}
locked: true
name: Evening
owner: IneFZ4CIEdSQQN4oCGExhsi0cWquxMrZY6tEElKM
picture: ''
recycle: false
version: 2

